Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Distributed Tensorflow with Horovod
In this tutorial, you will train a word2vec model in TensorFlow using distributed training via [Horovod](https://github.com/uber/horovod).

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.2


## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [2]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [3]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /home/nbuser/library/aml_config/config.json
Workspace name: AMLSworkspace
Azure region: westeurope
Subscription id: 70b8f39e-8863-49f7-b6ba-34a80799550c
Resource group: resgrpAMLS


## Create a remote compute target
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) to execute your training script on. In this tutorial, you create an `AmlCompute` cluster as your training compute resource. This code creates a cluster for you if it does not already exist in your workspace.

**Creation of the cluster takes approximately 5 minutes.** If the cluster is already in your workspace this code will skip the cluster creation process.

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

Found existing compute target
{'allocationState': 'Resizing', 'allocationStateTransitionTime': '2018-12-26T13:37:22.850000+00:00', 'creationTime': '2018-12-10T16:09:58.703952+00:00', 'currentNodeCount': 2, 'errors': [{'code': 'ClusterCoreQuotaReached', 'message': 'Operation results in exceeding quota limits of Total Cluster Dedicated Regional vCPUs. Maximum allowed: 24, Current in use: 14, Additional requested: 12. Please contact support to increase the quota for resource type Total Cluster Dedicated Regional vCPUs', 'error': {'code': 'ClusterCoreQuotaReached', 'message': 'Operation results in exceeding quota limits of Total Cluster Dedicated Regional vCPUs. Maximum allowed: 24, Current in use: 14, Additional requested: 12. Please contact support to increase the quota for resource type Total Cluster Dedicated Regional vCPUs'}}], 'modifiedTime': '2018-12-10T16:11:24.446527+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount

The above code creates a GPU cluster. If you instead want to create a CPU cluster, provide a different VM size to the `vm_size` parameter, such as `STANDARD_D2_V2`.

## Upload data to datastore
To make data accessible for remote training, AML provides a convenient way to do so via a [Datastore](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data). The datastore provides a mechanism for you to upload/download data to Azure Storage, and interact with it from your remote compute targets. 

If your data is already stored in Azure, or you download the data as part of your training script, you will not need to do this step. For this tutorial, although you can download the data in your training script, we will demonstrate how to upload the training data to a datastore and access it during training to illustrate the datastore functionality.

First, download the training data from [here](http://mattmahoney.net/dc/text8.zip) to your local machine:

In [5]:
import os
import urllib

os.makedirs('./data', exist_ok=True)
download_url = 'http://mattmahoney.net/dc/text8.zip'
urllib.request.urlretrieve(download_url, filename='./data/text8.zip')

('./data/text8.zip', <http.client.HTTPMessage at 0x7fc26e89c780>)

Each workspace is associated with a default datastore. In this tutorial, we will upload the training data to this default datastore.

In [6]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureBlob amlsworkspace9663571855 azureml-blobstore-4446f3c5-7943-4ab5-b1f3-5e4c7ede791c


Upload the contents of the data directory to the path `./data` on the default datastore.

In [7]:
ds.upload(src_dir='data', target_path='data', overwrite=True, show_progress=True)

$AZUREML_DATAREFERENCE_baa221bf55dc4ef5abfc62c70a0ff449

For convenience, let's get a reference to the path on the datastore with the zip file of training data. We can do so using the `path` method. In the next section, we can then pass this reference to our training script's `--input_data` argument. 

In [8]:
path_on_datastore = 'data/text8.zip'
ds_data = ds.path(path_on_datastore)
print(ds_data)

$AZUREML_DATAREFERENCE_3c18fcc578b242b1a5baa22c0860b576


## Train model on the remote compute

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [9]:
import os

project_folder = './tf-distr-hvd'
os.makedirs(project_folder, exist_ok=True)

Copy the training script `tf_horovod_word2vec.py` into this project directory.

In [10]:
import shutil

shutil.copy('tf_horovod_word2vec.py', project_folder)

'./tf-distr-hvd/tf_horovod_word2vec.py'

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed TensorFlow tutorial. 

In [11]:
from azureml.core import Experiment

experiment_name = 'tf-distr-hvd'
experiment = Experiment(ws, name=experiment_name)

### Create a TensorFlow estimator
The AML SDK's TensorFlow estimator enables you to easily submit TensorFlow training jobs for both single-node and distributed runs. For more information on the TensorFlow estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-tensorflow).

In [12]:
from azureml.train.dnn import TensorFlow

script_params={
    '--input_data': ds_data
}

estimator= TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='tf_horovod_word2vec.py',
                      node_count=2,
                      process_count_per_node=1,
                      distributed_backend='mpi',
                      use_gpu=True)

The above code specifies that we will run our training script on `2` nodes, with one worker per node. In order to execute a distributed run using MPI/Horovod, you must provide the argument `distributed_backend='mpi'`. Using this estimator with these settings, TensorFlow, Horovod and their dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `TensorFlow` constructor's `pip_packages` or `conda_packages` parameters.

Note that we passed our training data reference `ds_data` to our script's `--input_data` argument. This will 1) mount our datastore on the remote compute and 2) provide the path to the data zip file on our datastore.

### Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [13]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: tf-distr-hvd,
Id: tf-distr-hvd_1545831657015,
Type: azureml.scriptrun,
Status: Queued)


### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [14]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

Alternatively, you can block until the script has completed training before running more code.

In [15]:
run.wait_for_completion(show_output=True)

RunId: tf-distr-hvd_1545831657015

Streaming azureml-logs/80_driver_log_rank_0.txt

the input data is at /mnt/batch/tasks/shared/LS_root/jobs/amlsworkspace/azureml/tf-distr-hvd_1545831657015/mounts/workspaceblobstore/data/text8.zip
Use the data from /mnt/batch/tasks/shared/LS_root/jobs/amlsworkspace/azureml/tf-distr-hvd_1545831657015/mounts/workspaceblobstore/data/text8.zip
Found and verified /mnt/batch/tasks/shared/LS_root/jobs/amlsworkspace/azureml/tf-distr-hvd_1545831657015/mounts/workspaceblobstore/data/text8.zip
Data size 17005207
Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']
16 five -> 9 nine
16 five -> 4 one
4 one -> 16 five
4 one -> 2316 featuring
2316 featuring -> 1 the
2316 featuring -> 4 one
1 the -> 2316 featuring
1 the -> 299 best
Instructions for updating:
keep_

{'runId': 'tf-distr-hvd_1545831657015',
 'target': 'gpucluster',
 'status': 'Finalizing',
 'startTimeUtc': '2018-12-26T13:56:25.844486Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': 'a99248cd-9630-49d6-ace7-02702182e861'},
 'runDefinition': {'Script': 'tf_horovod_word2vec.py',
  'Arguments': ['--input_data',
   '$AZUREML_DATAREFERENCE_3c18fcc578b242b1a5baa22c0860b576'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 3,
  'Target': 'gpucluster',
  'DataReferences': {'3c18fcc578b242b1a5baa22c0860b576': {'DataStoreName': 'workspaceblobstore',
    'Mode': 'Mount',
    'PathOnDataStore': 'data/text8.zip',
    'PathOnCompute': None,
    'Overwrite': False}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 2,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['pyth